# Computer Vision Lab Project - JKU - 2021
### Team B0
  
Team Members: Dominik Heindl, Philipp Eberstaller, Carson Wittwer  

In [4]:
from abc import ABC, abstractmethod
import glob
import json
import os
import sys
from time import time, gmtime, strftime
from typing import Tuple, Dict, List

import cv2
import matplotlib.pyplot as plt
import numpy as np
import spectral as sp
from scipy.stats import chi2

import torch
from torchvision.io import write_video
from torchvision.transforms import functional, ToTensor, Lambda, Compose
from torchvision.utils import save_image

import utils as ut

from clahe import clahe

#### Loading Data Utilities

In [62]:
def load_homographies(input_dir: str):
    """
    Finds all homographies.json files recursively in a given input directory.

    Args:
        input_dir: Input directory which contains .json files

    Returns:
        A dictionary mapping the folder names to dictionaries with the homographies
        Each dictionary is structured as follows:
            {Folder Name (e.g. "valid-1-0"): {File Name (e.g. 0-B01): List[homographies]}}}
    """
    sorted_homographies = sorted(glob.glob(
        os.path.join(input_dir, '**', '*ies.json'),
        recursive=True
    ))

    homographies = {}

    for file in sorted_homographies:
        folders, file_name = os.path.split(file)
        folder_name = os.path.basename(folders)

        homographies[folder_name] = json.loads(open(file, 'r').read())

    return homographies


def read_images(
        input_dir: str,
        mask: np.ndarray,
        integrated_only: bool = True
) -> Tuple[Dict[str, Dict[str, Dict[str, np.ndarray]]], Dict[str, Dict[str, Dict[str, np.ndarray]]]]:
    """
    Finds all .png files recursively in a given input directory. The images are loaded,
    a mask is applied to filter unwanted regions of the images and the images get
    warped based on the provided homographies.
    With this dictionary structure it can be assured that the images of one
    image set always go together.

    Args:
        input_dir: Input directory which contains .png images
        mask: Filter to mask out unwanted regions of all images at the same position
        integrated_only: Whether to load only the integrated

    Returns:
        A tuple containing 2 dictionaries for the original and warped images.
        Each dictionary is structured as follows:
            {Folder Name (e.g. "valid-1-0"): {File Set Number (e.g. 0): {File Name (e.g. 0-B01): Image as array}}}
    """
    input_dir = os.path.abspath(input_dir)

    if integrated_only:
        sorted_files = sorted(glob.glob(
            os.path.join(input_dir, '**', '*-int_image.png'),
            recursive=True
        ))
    else:
        sorted_files = sorted(glob.glob(
            os.path.join(input_dir, '**', '*.png'),
            recursive=True
        ))

        homographies = load_homographies(input_dir)

    original_images = {}
    warped_images = {}
    
    for file in sorted_files:
        
        folders, file_name = os.path.split(file)
        
        if not integrated_only:
            if any(x not in file_name for x in ('mask', 'int_image', 'mask_image')):
                continue

        file_name = file_name.split('.')[0]
        folder_name = os.path.basename(folders)
        file_set = file_name.split('-')[0]

        if folder_name not in original_images and folder_name not in warped_images:
                original_images[folder_name] = {}
                warped_images[folder_name] = {}

        if file_set not in original_images[folder_name] and file_set not in warped_images[folder_name]:
            original_images[folder_name].update({file_set: {}})
            warped_images[folder_name].update({file_set: {}})

        img = cv2.imread(file)
        
        if not integrated_only:

            if folder_name not in warped_images:
                warped_images[folder_name] = {}

            if file_set not  in warped_images[folder_name]:
                warped_images[folder_name].update({file_set: {}})

            w, h, _ = img.shape
            img = cv2.bitwise_and(img,img,mask=mask) #apply the mask to remove the timestamp

            homography = np.array(homographies[folder_name][file_name]) #get homography

            warped_img = cv2.warpPerspective(img, homography, (w,h)) #warp the image using the given homography
            warped_images[folder_name][file_set].update({file_name: warped_img})

        original_images[folder_name][file_set].update({file_name: img}) #save the original

    return original_images, warped_images

#### Other Utilities


In [27]:
def display_images(images, figsize=None, resize_dim=(200, 200)):
    if figsize: row, col = figsize
    else: row, col = 1,1
        
    fig, axs = plt.subplots(row, col, figsize=figsize)
    if row>1 or col>1: axs = axs.flatten()
    else: axs = [axs]
    fig.set_figwidth(20)
    fig.set_figheight(15)
    
    for img, ax in zip(images, axs):
        ax.imshow(cv2.resize(img, resize_dim), interpolation='nearest')
        
    plt.show()

def integrate(images: np.ndarray):

    integrated_images = []
    mask_images = []
    background_images = []
    """
    This is going through all the sets of images,True
    where images[0] represents 10 images in a numpy array
    """
    for i in range(len(images)): 
        integral = np.zeros((images[0].shape[1:]), np.float64)
        mask = np.zeros((images[0].shape[1:]), np.float64)
        mask = mask[:,:,0]
        mask_ = np.copy(mask)

        for j in range(len(images[i])):
            integral += images[i][j]
            
            height, width, _ = images[i][j].shape
            for x in range(height):
                for y in range(width):
                    if all(images[i][j][x,y] == [0,0,0]):
                        mask[x,y] += 1
        
        x,y = mask.shape
        for n in range(x):
            for m in range(y):
                if mask[n,m] == 0:
                    mask_[n,m] = 1
                else:
                    mask_[n,m] = 0
        
        #cv2.cvtColor(np.float32(mask_),cv2.COLOR_GRAY2BGR)

        mask_images.append(mask_)
        integral /= len(images[i]) 
        integral=cv2.normalize(integral, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        integrated_images.append(integral)

    return integrated_images , mask_images

def color_check(image: np.ndarray):
    height, width = image.shape[:2]
    cc = np.zeros(image.shape, np.float64)
    for y,x in zip(range(height),range(width)):
        if all(image[x,y] != [0,0,0]):
            cc[x,y]=[0,0,0]
    return np.array(cc)

def crop_image(image):
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    _,thresh = cv2.threshold(grayscale,0,255,cv2.THRESH_OTSU)

    bbox = cv2.boundingRect(thresh)
    x,y,w,h = bbox
    
    return x,y,w,h

## Anomaly Detection

I think it is good if we use a proper base class for the anomaly detection, this should ensure a unified interface which can be used at any point in our pipeline. Basically any algorithm implementation / class should have the method `anomaly_mask` which should return a mask where outliers are marked with 1 and other pixels remain 0.

In [5]:
class AnomalyDetection(ABC):
    def __init__(self, img: np.ndarray):
        self.img = img

    @abstractmethod
    def anomaly_mask(self) -> np.ndarray:
        """
        Takes whatever is necessary from the current object to generate an anomaly
        mask. The mask has the same height and width as the input image, anomalies are
        marked as 1, everything als with 0.
        """
        pass

### Reed-Xiaoli Detector
This anomaly detection algorithm is the baseline, which should not be used. Nevertheless it is good to have it implemented for comparisons.

In [6]:
class ReedXiaoliAnomaly(AnomalyDetection):
    def __init__(
            self,
            img: np.ndarray,
            conf_coef: float = 0.999,
            verbose: bool = False
    ):
        super().__init__(img)
        self.conf_coef = conf_coef
        self.verbose = verbose
        self.D_rx = None
        self.mask = None

    def _rxd(self):
        """
        Implementation of the Reed-Xiaoli detector. Calculates a score for each pixel as the
        Mahalanobis distance between the pixel and the background.

        Args:
            img: Image in the shape Height x Width x 3

        Returns:
            Numpy array with the Mahalanobis distance of each pixel with shape Height x Width of the input image
        """
        h, w, c = self.img.shape

        # Calculate Mean Vector
        mean_vec = np.mean(img, axis=(0, 1))

        # Calculate Covariance Matrix
        img_tmp = img.copy()
        img_tmp = img_tmp.astype('float32') - mean_vec
        img_tmp = img_tmp.reshape(-1, 3)
        cov_mat = (img_tmp.T @ img_tmp) / (img_tmp.shape[0] - 1)

        # Initialize Result Matrix
        D_rx = np.zeros((h,w))

        # Calculate Mahalanobis distance
        for i in range(h):
            for j in range(w):
                r = img[i,j]
                r_normal = r - mean_vec
                D_rx[i,j] = r_normal.T @ np.linalg.inv(cov_mat) @ r_normal

        return D_rx

    def anomaly_mask(self) -> np.ndarray:
        """
        Calculates a threshold for the values of the RX-detector in order to find
        only the most extreme anomalies which have a higher RX-score than conf_coef %
        of the values in the array. All values below this threshold are set to 0.
        If verbose is True, then the cumulative probability density of the rxd_array is
        printed alongside the returned mask.

        Returns:
            Anomaly mask where anomalies are 0, everything else is 0
        """
        self.D_rx = self._rxd()
        rxd_arr = self.D_rx.copy()
        rxd_sorted = np.sort(rxd_arr.flatten())
        prob = 1. * np.arange(len(rxd_sorted)) / (len(rxd_sorted) - 1)

        threshold_idx = np.argwhere(prob > self.conf_coef)[0][0]
        threshold = rxd_sorted[threshold_idx]

        mask = np.where(rxd_arr > threshold, 1, 0)
        # mask = rxd_arr > threshold
        # rxd_arr[mask] = 1
        # rxd_arr[~mask] = 0

        self.mask = mask

        if self.verbose:
            fig = plt.figure(figsize=(15,7))
            ax1 = fig.add_subplot(121)
            ax1.plot(rxd_sorted, prob)
            ax1.title.set_text("CDF of RX-Values")
            ax1.set_xlabel("RX Value")
            ax1.set_ylabel("Cumulative Probability")

            ax2 = fig.add_subplot(122)
            ax2.imshow(rxd_arr)
            ax2.title.set_text("Anomaly Mask")

        return self.mask

### Residual Approach

Originally this should be an implementation from `Reducing Anomaly Detection in Images to Detection in Noise` by Davy et al. The implementation is heavily influenced from this [GitHub Repository](https://github.com/whynotw/reducing_anomaly_detection), however by pure chance I stumbled across the fact that a burred version of the residual image is already looking promising for detection outliers. <br>
A disadvantage is that this algorithm takes ~1 min. per image to create the anomaly mask!

In [16]:
class ResidualAnomaly(AnomalyDetection):
    def __init__(
            self,
            img: np.ndarray,
            filter_size: int = 7,
            stride: int = 3,
            number_k: int = 5,
            threshold_intensity: int = 9
    ):
        super().__init__(img)
        if filter_size % 2 == 0:
            print("size of filter %d should be odd number" % filter_size)
        self.filter_size = filter_size
        self.stride = stride
        self.k = number_k
        self.threshold_intensity = threshold_intensity

        self.image_origin = img
        h, w, _ = img.shape

        h_remain = (h - filter_size) % stride
        w_remain = (w - filter_size) % stride
        self.image_origin = self.image_origin[:h - h_remain, :w - w_remain, :]

        self.h, self.w, _ = self.image_origin.shape
        self.h_red = (self.h - self.filter_size) // self.stride + 1
        self.w_red  = (self.w - self.filter_size) // self.stride + 1
        self.dim_data = 3 * self.filter_size ** 2
        self.img_acc = np.zeros((self.h, self.w, 3), dtype=np.float32)
        self.pixel_count = np.zeros((self.h, self.w), dtype=np.uint8)

        tmp_idx = np.arange(0, self.w_red * self.stride, self.stride, dtype=np.uint16)
        self.x0_data = np.stack((tmp_idx, ) * self.h_red)
        self.x1_data = self.x0_data+self.filter_size

        tmp_idx = np.arange(0, self.h_red * self.stride, self.stride, dtype=np.uint16)
        self.y0_data = np.stack((tmp_idx, ) * self.w_red).T
        self.y1_data = self.y0_data+self.filter_size
        self.knn = cv2.ml.KNearest_create()

        self.mask = None

    def _create_patches(self):
        """
        Divides the given image into pathes with size filter_size x filter_size.
        The filter is then moves forward for stride pixels. All patches are saved
        in the data object for the knn algorithm.

        Returns:
            None
        """
        self.data_total  = np.empty((self.h_red, self.w_red, self.dim_data), dtype=np.uint8)
        self.label_total = np.empty((self.h_red, self.w_red, 1), dtype=np.float32)

        count = 0
        for j in range(0, self.h_red):
            for i in range(0, self.w_red):
                x0 = i * self.stride
                x1 = x0 + self.filter_size
                y0 = j * self.stride
                y1 = y0 + self.filter_size
                self.data_total[j, i, :] = self.image_origin[y0:y1, x0:x1, :].reshape(-1)
                self.label_total[j, i, 0] = count
                count += 1

    def _get_knn(self):
        """
        Performs a knn algorithm on the previously calculated patches. This should
        ensure that anomalies are true anomalies and not part of some bigger pattern

        Returns:
            None
        """
        tmp_data = self.data_total.reshape((-1, self.dim_data)).astype(np.float32)

        self.knn.train(
            tmp_data,
            cv2.ml.ROW_SAMPLE,
            self.label_total.reshape(-1,1)
        )

        _, _, neighbours, distances = self.knn.findNearest(
            tmp_data,
            self.filter_size ** 2 + self.k
        )
        neighbours = neighbours.astype(np.int64)

        return neighbours, distances

    def _get_self_similar_estimate(self):
        """
        Creates a "self-similar" version of each image patch which is later used
        to create the residual image. The average values of the neighbour patches
        are divided from the current patch.

        Returns:
            None
        """
        neighbours,distances = self._get_knn()

        count = 0
        for j in range(0, self.h_red):
            for i in range(0, self.w_red):
                patch = np.zeros((self.filter_size, self.filter_size, 3), dtype=np.float32)
                x0 = self.stride * i
                x1 = x0 + self.filter_size
                y0 = self.stride * j
                y1 = y0 + self.filter_size

                partition = 1e-7
                effective = -1

                for k in range(len(neighbours)):
                    n = neighbours[count][k]
                    j_neighbour = n // self.w_red
                    i_neighbour = n % self.w_red

                    d = distances[count][k]

                    x0_n = self.stride * i_neighbour
                    x1_n = x0_n + self.filter_size
                    y0_n = self.stride * j_neighbour
                    y1_n = y0_n + self.filter_size

                    if x0 > x1_n or x1 < x0_n or y0 > y1_n or y1 < y0_n:
                        effective += 1
                    else:
                        continue

                    if effective == 0:
                        d0 = d

                    if effective >= self.k:
                        break

                    d -= d0
                    factor = np.exp(-d/self.dim_data)

                    if factor > 2**-8:
                        patch += factor * (
                            self.data_total[j_neighbour,i_neighbour,:]
                                .reshape((self.filter_size, self.filter_size, 3))
                        )
                        partition += factor

                self.img_acc[y0:y1, x0:x1, :] += patch / partition
                self.pixel_count[y0:y1, x0:x1] += 1
                count += 1

        self.img_red = np.uint8(self.img_acc / np.stack((self.pixel_count,) * 3, axis=-1))

    def anomaly_mask(self):
        """
        Calculates the residual image -> original image - self similar version of it.
        Then apply a GaussianBlus which removes weak outlier signals. The blurred image
        is then filtered for the top outlier value > some threshold.

        Returns:
            Anomaly mask where anomalies are 0, everything else is 0
        """
        self._create_patches()
        self._get_self_similar_estimate()
        tmp_residual = np.uint8(
            np.sum(
                np.abs(
                    np.int16(self.img_red) - np.int16(self.image_origin)
                ),
                axis=-1
            ) / 3
        )

        tmp_blurred = cv2.GaussianBlur(
            tmp_residual,
            (self.filter_size * 2 + 1,) * 2,
            0
        )

        mask = np.where(tmp_blurred >= self.threshold_intensity, 1, 0)
        self.mask = mask

        return self.mask

## Bounding Boxes

In [ ]:
with open(os.path.join(IMAGE_FOLDER, 'labels.json'), 'r') as label_file:
    gtlabels = json.load(label_file)

val_img = cv2.imread('data/validation/valid-1-0/3-B01.png')

### Add given BB

In [91]:
def add_bounding_boxes(
        img: np.ndarray,
        gtlabels: List[List[int]],
        rgb_color: Tuple[int] = (0, 0, 255),
        thickness: int = 5
) -> np.ndarray:
    """
    Adds the given bounding boxes to an image. These bounding boxes are
    only aligned for the center image!

    Args:
        img: Image to which bounding boxes should be added
        gtlabels: Bounding boxes from a labels.json file
        rgb_color: Tuple representing the RGB color of the box
        thickness: Thickness of the box line

    Returns:
        Image with an overlayed bounding boxes
    """
    for bb in gtlabels:
        x, y, w, h = bb
        img = cv2.rectangle(
            img=img,
            pt1=(x,y),
            pt2=(x+w, y+h),
            color=rgb_color,
            thickness=thickness
        )

    return img

### Create new BB from anomaly mask

In [92]:
def is_close(cnt1,cnt2, max_dist=30):
    row1,row2 = cnt1.shape[0],cnt2.shape[0]
    for i in range(row1):
        for j in range(row2):
            dist = np.linalg.norm(cnt1[i]-cnt2[j])
            if abs(dist) < max_dist:
                return True
            elif i==row1-1 and j==row2-1:
                return False

def combine_contours(contours, min_area):
    n = len(contours)
    status = np.zeros((n,1))
    
    for i,cnt1 in enumerate(contours):
        x = i    
        if i != n-1:
            for j,cnt2 in enumerate(contours[i+1:]):
                x = x+1
                dist = is_close(cnt1,cnt2)
                if dist:
                    val = min(status[i],status[x])
                    status[x] = status[i] = val
                else:
                    if status[x]==status[i]:
                        status[x] = i+1

    unified = []
    maximum = int(status.max())+1
    for i in range(maximum):
        pos = np.where(status==i)[0]
        if pos.size != 0:
            cont = np.vstack(contours[i] for i in pos)
            hull = cv2.convexHull(cont)
            if not cv2.contourArea(hull)< min_area:
                unified.append(hull)
                
    return unified

def area_filled(contour,mask):
    
    contour_area=np.zeros_like(mask)
    cv2.drawContours(contour_area, [contour], -1, 1, -1)
    
    total=contour_area.sum()
    mask_area=contour_area[mask.astype(bool)].sum()

    return mask_area.sum()/total

In [93]:
def get_bounding_boxes(image, mask, min_area=100, fill_threshold=0.3):
    
    contours = cv2.findContours(image.astype(np.uint8), mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)[0]
    
    if not len(contours):
        detected_anomalies_bb = np.zeros((0,4))
        
        return detected_anomalies_bb, contours
        
    combined=combine_contours(contours, min_area)
    combined = [c for c in combined if not area_filled(c,mask)<fill_threshold]
    
    if not len(combined):
        detected_anomalies_bb = np.zeros((0,4))
        return detected_anomalies_bb, combined
    
    detected_anomalies_bb = np.stack([cv2.boundingRect(c) for c in combined])
    
    return detected_anomalies_bb, combined

##### Lucas Kanade Optical Flow
Doesn't work...

In [135]:
''' 
    inRange checks whether the given cordinates line in the given image limits
    cordinates, limits are tuples i.e., (X,Y) 
'''
def inRange( cordinates, limits):
	x,y = cordinates
	X_Limit, Y_Limit = limits
	return 0 <= x and x < X_Limit and 0 <= y and y < Y_Limit

'''
    opticalFlow calculates the displacements in X and Y directions i.e., (u,v)
    given two consecutive images varying with time
'''
def optical_flow(old_frame, new_frame, window_size, min_quality=0.01):

    max_corners = 10000
    min_distance = 0.1
    feature_list = cv2.goodFeaturesToTrack(old_frame, max_corners, min_quality, min_distance)

    w = int(window_size/2)

    old_frame = old_frame / 255
    new_frame = new_frame / 255

    #Convolve to get gradients w.r.to X, Y and T dimensions
    kernel_x = np.array([[-1, 1], [-1, 1]])
    kernel_y = np.array([[-1, -1], [1, 1]])
    kernel_t = np.array([[1, 1], [1, 1]])

    fx = cv2.filter2D(old_frame, -1, kernel_x)              #Gradient over X
    fy = cv2.filter2D(old_frame, -1, kernel_y)              #Gradient over Y
    ft = cv2.filter2D(new_frame, -1, kernel_t) - cv2.filter2D(old_frame, -1, kernel_t)  #Gradient over Time


    u = np.zeros(old_frame.shape)
    v = np.zeros(old_frame.shape)

    for feature in feature_list:        #   for every corner
            j, i = feature.ravel()		#   get cordinates of the corners (i,j). They are stored in the order j, i
            i, j = int(i), int(j)		#   i,j are floats initially

            I_x = fx[i-w:i+w+1, j-w:j+w+1].flatten()
            I_y = fy[i-w:i+w+1, j-w:j+w+1].flatten()
            I_t = ft[i-w:i+w+1, j-w:j+w+1].flatten()

            b = np.reshape(I_t, (I_t.shape[0],1))
            A = np.vstack((I_x, I_y)).T

            U = np.matmul(np.linalg.pinv(A), b)     # Solving for (u,v) i.e., U

            u[i,j] = U[0][0]
            v[i,j] = U[1][0]

    return (u,v)


'''
Draw the displacement vectors on the image, given (u,v) and save it to the output filepath provided
'''
def drawOnFrame(frame, U, V, output_file):

    line_color = (0, 255, 0) #  Green

    for i in range(frame.shape[0]):
        for j in range(frame.shape[1]):
            u, v = U[i][j], V[i][j]

            if u and v:
                frame = cv2.arrowedLine( frame, (i, j), (int(round(i+u)), int(round(j+v))),
                                        (0, 255, 0),
                                        thickness=1
                                    )
    cv2.imwrite(output_file, frame)


'''
Create a plot of the displacement vectors given (u,v) and plot the two images and displacement in a row.
Save the plot to given output filepath
'''
def drawSeperately(old_frame, new_frame, U, V):

    displacement = np.ones_like(new_frame)
    displacement.fill(255.)             #Fill the displacement plot with White background
    line_color =  (0, 0, 0)
    # draw the displacement vectors
    for i in range(new_frame.shape[0]):
        for j in range(new_frame.shape[1]):

            start_pixel = (i,j)
            end_pixel = ( int(i+U[i][j]), int(j+V[i][j]) )

            #check if there is displacement for the corner and endpoint is in range
            if U[i][j] and V[i][j] and inRange( end_pixel, old_frame.shape ):     
                displacement = cv2.arrowedLine( displacement, start_pixel, end_pixel, line_color, thickness =2)

    fig = plt.figure(figsize=(15,7))
    ax1 = fig.add_subplot(131)
    ax1.imshow(old_frame)
    ax1.title.set_text("First Image")

    ax2 = fig.add_subplot(132)
    ax2.imshow(new_frame)
    ax2.title.set_text("Second Image")

    ax3 = fig.add_subplot(133)
    ax3.imshow(displacement)
    ax3.title.set_text("Displacements")
    fig.tight_layout()

## Working Locally 

Working on using the temporal data

In [ ]:
MASK = 'data/mask.png'
IMAGE_FOLDER = 'data/validation'

mask = cv2.imread(MASK,0)

orig_images_, warped_images_ = read_images(IMAGE_FOLDER, mask,integrated_only=False)

img_lists = {}

for img_set, time_step_dict in warped_images_.items():
    img_lists[img_set] = []
    for time_step, n_camera_dict in time_step_dict.items():
        images = []
        for cam, img in n_camera_dict.items():
            images.append(img)
        img_lists[img_set].append(np.array(images))


for folder, list in img_lists.items():
    int_images, mask_images = integrate(list)
    display_images(int_images, figsize=(7,1), resize_dim=(200,200))
    display_images(mask_images, figsize=(7,1), resize_dim=(200,200))
    if True: exit


In [3]:
def get_background(images):
    return np.median(images, axis=0).astype(np.uint8)

In [28]:
MASK = 'data/mask.png'
IMAGE_FOLDER = 'data_combined/validation/valid-1-0/'
BB_LABELS_FOLDER = 'data/validation/valid-1-0/'

mask = cv2.imread(MASK,0)
#homographies = json.loads(open(IMAGE_FOLDER+'homographies.json','r').read())

integrated_images = []
mask_images = []
anomaly_masks = []

for root, dirs, files in os.walk(IMAGE_FOLDER):
    for file in files:
        if 'int_image' in file:
            integrated_images.append(cv2.imread(os.path.join(root,file)))
        elif 'mask_image' in file:
            mask_images.append(cv2.imread(os.path.join(root,file)))
        elif 'anomaly_mask' in file:
            anomaly_masks.append(cv2.imread(os.path.join(root,file)))

#display_images(np.array(integrated_images), figsize=(1,7), resize_dim=(200,200))
#display_images(np.array(mask_images), figsize=(1,7), resize_dim=(200,200))
#display_images(np.array(anomaly_masks), figsize=(1,7), resize_dim=(200,200))

with open(os.path.join(BB_LABELS_FOLDER, 'labels.json'), 'r') as label_file:
    gtlabels = json.load(label_file)

with_valid_labels = []
with_our_labels = []
"""
for i in range(len(integrated_images)):
    with_valid_bbs = add_bounding_boxes(integrated_images[i],gtlabels)
    with_valid_labels.append(with_valid_bbs)

    our_bbs, thing = get_bounding_boxes(cv2.cvtColor(anomaly_masks[i],cv2.COLOR_BGR2GRAY),mask_images[i])
    with_our_bbs = add_bounding_boxes(integrated_images[i],our_bbs.tolist(),rgb_color=(255,0,0))
    with_our_labels.append(with_our_bbs)

    #fig = plt.figure(figsize=(15,7))
    #ax1 = fig.add_subplot(121)
    #ax1.imshow(with_valid_bbs)
    #ax1.title.set_text("Integrated Image BBs")
"""

for i in range(len(integrated_images)-1):
    im1 = cv2.cvtColor(integrated_images[i], cv2.COLOR_BGR2GRAY)
    im2 = cv2.cvtColor(integrated_images[i+1], cv2.COLOR_BGR2GRAY)

In [ ]:
tensorizer = ToTensor()

brightness_factor = 0.3
contrast_factor = 4
bri_cont_transform = Compose([
    Lambda(lambda x: functional.adjust_brightness(x, brightness_factor)),
    Lambda(lambda x: functional.adjust_contrast(x, contrast_factor))])

# bri_cont_transform = ColorJitter(brightness=0.4, contrast=3)
inverter_transform = Lambda(lambda x: functional.invert(x))

ims = integrated_images.copy()

inv = []

for i in range(len(ims)):
    ims[i] = tensorizer(ims[i])
    bri_cont = bri_cont_transform(ims[i])
    inverted = inverter_transform(bri_cont)
    temp=inverted.permute(1, 2, 0).numpy()
    inv.append(temp)

display_images(inv, figsize=(1,7), resize_dim=(200,200))

new_warps = []
for j in range(len(inv)):
    new_warps.append(cv2.cvtColor(inv[j], cv2.COLOR_BGR2GRAY))
ims = np.vstack([im.reshape(1,im.shape[0] * im.shape[1]) for im in new_warps])
# Compute the median column by column and reshape to the original shape:
back_img = np.median(ims, axis=0).reshape(1024,1024)

#inv.append(cv2.cvtColor(back_img,cv2.COLOR_GRAY2BGR))

#display_images(inv, figsize=(1,8), resize_dim=(200,200))

## Save Integrated Images, Anomaly Masks, etc.

Run the following section only if you have a lot of time :^)

In [ ]:
MASK = 'data/mask.png'
IMAGE_FOLDER = 'data_combined/validation/'

mask = cv2.imread(MASK,0)

orig_images_, warped_images_ = read_images(IMAGE_FOLDER, mask)

In [ ]:
def integrate_and_save(warped_images: Dict[str, Dict[str, Dict[str, np.ndarray]]], target_dir: str) -> None:
    """
    Takes the loaded and warped images, integrates the different camera views and saves the
    integrated images in the provided directory.
    Folder structure already has to be present!

    Args:
        warped_images: Warped images from the read_images function.
        target_dir: Directory where the integrated images + integration mask should be saved

    Returns:
        None
    """
    img_lists = {}

    for img_set, time_step_dict in warped_images.items():
        img_lists[img_set] = []
        for time_step, n_camera_dict in time_step_dict.items():
            images = []
            for cam, img in n_camera_dict.items():
                images.append(img)
            img_lists[img_set].append(np.array(images))


    for folder, list in img_lists.items():
        if len(os.listdir(os.path.join(target_dir, folder))) > 0:
            continue
        start = time()
        print(folder)
        int_images, mask_images = integrate(list)
        for i, (int, mas) in enumerate(zip(int_images, mask_images)):
            cv2.imwrite(os.path.join(target_dir, folder, f'{i}-int_image.png'), int)
            cv2.imwrite(os.path.join(target_dir, "train", folder, f'{i}-mask_image.png'), mas * 255)
        print(f"Time it took to integrate images from {folder}: {strftime('%H:%M:%S', gmtime(time()-start))}")

In [ ]:
for folder, time_step_dict in orig_images_.items():
    for time_step, img_dict in time_step_dict.items():
        for _, img in img_dict.items():
            start = time()
            reducing = ResidualAnomaly(img=img)
            red_mask = reducing.anomaly_mask()
            cv2.imwrite(os.path.join("data_combined", "validation", folder, f'{time_step}-anomaly_mask.png'), red_mask * 255)
            print(f"It took {strftime('%H:%M:%S', gmtime(time()-start))}")